## Heavily adapted from 
[here](https://github.com/pytorch/examples/blob/master/vae/main.py)

In [15]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
batch_size=128
epochs = 1000
log_interval = 100

In [18]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.ToTensor()
                  ),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [19]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [20]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [25]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta=1):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    #L1 = F.l1_loss(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

   # return L1 + beta * KLD   
    return BCE + beta * KLD

In [26]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar, beta=2)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [27]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [28]:
## Main
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    if epochs % 10 == 0:
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 547.596191
Train Epoch: 1 [12800/60000 (21%)]	Loss: 189.102432
Train Epoch: 1 [25600/60000 (43%)]	Loss: 174.724045
Train Epoch: 1 [38400/60000 (64%)]	Loss: 156.869171
Train Epoch: 1 [51200/60000 (85%)]	Loss: 145.890823
====> Epoch: 1 Average loss: 177.7591
====> Test set loss: 133.5746
Train Epoch: 2 [0/60000 (0%)]	Loss: 150.440979
Train Epoch: 2 [12800/60000 (21%)]	Loss: 140.160583
Train Epoch: 2 [25600/60000 (43%)]	Loss: 137.942596
Train Epoch: 2 [38400/60000 (64%)]	Loss: 137.590393
Train Epoch: 2 [51200/60000 (85%)]	Loss: 142.511902
====> Epoch: 2 Average loss: 141.4514
====> Test set loss: 120.6297
Train Epoch: 3 [0/60000 (0%)]	Loss: 136.574265
Train Epoch: 3 [12800/60000 (21%)]	Loss: 134.810165
Train Epoch: 3 [25600/60000 (43%)]	Loss: 132.594711
Train Epoch: 3 [38400/60000 (64%)]	Loss: 135.467987
Train Epoch: 3 [51200/60000 (85%)]	Loss: 133.522247
====> Epoch: 3 Average loss: 134.7602
====> Test set loss: 116.3437
Train Epoch: 4 [0/60000 (0%)]	L

KeyboardInterrupt: 

In [ ]:
28*28